In [1]:
from math import sqrt
import numpy as np
import sklearn
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestRegressor
import pandas as pd
import matplotlib.pyplot as plt
from numpy import concatenate
from matplotlib import pyplot
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from pandas import read_csv
from datetime import datetime
from keras.layers import Bidirectional
import datetime
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from numpy import array
import time

Using TensorFlow backend.


In [2]:
def rmse(pred, actual):
    error = np.subtract(pred, actual)
    sqerror= np.sum(np.square(error))/actual.shape[0]
    return np.sqrt(sqerror)

In [3]:
def MODEL_FNN_adam(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations):
    train_acc=np.zeros(Num_Exp)
    test_acc=np.zeros(Num_Exp)
    Step_RMSE=np.zeros([Num_Exp,n_steps_out])
    mlp_adam = MLPRegressor(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.1,max_iter=Iterations, tol=0)
    
    start_time=time.time()
    for run in range(Num_Exp):
        print("Experiment",run+1,"in progress")
        mlp_adam.fit(x_train, y_train)
        y_predicttrain = mlp_adam.predict(x_train)
        y_predicttest = mlp_adam.predict(x_test)
        train_acc[run] = rmse( y_predicttrain, y_train) 
        test_acc[run] = rmse( y_predicttest, y_test)
        for j in range(n_steps_out):
            Step_RMSE[run][j]=rmse(y_predicttest[:,j], y_test[:,j])
            
    print("Total time for",Num_Exp,"experiments",time.time()-start_time)
    return train_acc,test_acc,Step_RMSE

def MODEL_FNN_sgd(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations):
    train_acc=np.zeros(Num_Exp)
    test_acc=np.zeros(Num_Exp)
    Step_RMSE=np.zeros([Num_Exp,n_steps_out])
    mlp_sgd = MLPRegressor(hidden_layer_sizes=(100, ), activation='relu', solver='sgd', alpha=0.1,max_iter=Iterations, tol=0)
    
    start_time=time.time()
    for run in range(Num_Exp):
        print("Experiment",run+1,"in progress")
        mlp_sgd.fit(x_train,y_train)
        y_predicttrain = mlp_sgd.predict(x_train)
        y_predicttest = mlp_sgd.predict(x_test)
        train_acc[run] = rmse( y_predicttrain,y_train) 
        test_acc[run] = rmse( y_predicttest, y_test) 
        for j in range(n_steps_out):
            Step_RMSE[run][j]=rmse(y_predicttest[:,j], y_test[:,j])
            
    print("Total time for",Num_Exp,"experiments",time.time()-start_time)
    return train_acc,test_acc,Step_RMSE


def MODEL_LSTM(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations):
    n_features = 1
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
    print(x_train.shape)
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features))
    print(x_test.shape)
    
    train_acc=np.zeros(Num_Exp)
    test_acc=np.zeros(Num_Exp)
    Step_RMSE=np.zeros([Num_Exp,n_steps_out])
    
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps_in,n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    
    start_time=time.time()
    for run in range(Num_Exp):
        print("Experiment",run+1,"in progress")
        # fit model
        model.fit(x_train, y_train, epochs=Iterations,batch_size=64, verbose=0, shuffle=False)
        y_predicttrain = model.predict(x_train)
        y_predicttest = model.predict(x_test)
        train_acc[run] = rmse( y_predicttrain,y_train) 
        test_acc[run] = rmse( y_predicttest, y_test) 
        for j in range(n_steps_out):
            Step_RMSE[run][j]=rmse(y_predicttest[:,j], y_test[:,j])
            
    print("Total time for",Num_Exp,"experiments",time.time()-start_time)
    return train_acc,test_acc,Step_RMSE


def MODEL_Bi_LSTM(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations):
    n_features = 1
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
    print(x_train.shape)
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features))
    print(x_test.shape)
    
    train_acc=np.zeros(Num_Exp)
    test_acc=np.zeros(Num_Exp)
    Step_RMSE=np.zeros([Num_Exp,n_steps_out])
    
    model = Sequential()
    model.add(Bidirectional(LSTM(100, activation='relu'), input_shape=(n_steps_in,n_features)))
    model.add(Dense(n_steps_out))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    
    start_time=time.time()
    for run in range(Num_Exp):
        print("Experiment",run+1,"in progress")
        # fit model
        model.fit(x_train, y_train, epochs=Iterations,batch_size=64, verbose=0, shuffle=False)
        y_predicttrain = model.predict(x_train)
        y_predicttest = model.predict(x_test)
        train_acc[run] = rmse( y_predicttrain,y_train) 
        test_acc[run] = rmse( y_predicttest, y_test) 
        for j in range(n_steps_out):
            Step_RMSE[run][j]=rmse(y_predicttest[:,j], y_test[:,j])
            
    print("Total time for",Num_Exp,"experiments",time.time()-start_time)
    return train_acc,test_acc,Step_RMSE


def MODEL_EN_DC(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations):
    n_features = 1
    x_train = x_train.reshape((x_train.shape[0], x_train.shape[1], n_features))
    print(x_train.shape)
    x_test = x_test.reshape((x_test.shape[0], x_test.shape[1], n_features))
    print(x_test.shape)
    y_train = y_train.reshape((y_train.shape[0], y_train.shape[1], n_features))
    print(y_train.shape)
    y_test = y_test.reshape((y_test.shape[0], y_test.shape[1], n_features))
    print(y_test.shape)
    
    train_acc=np.zeros(Num_Exp)
    test_acc=np.zeros(Num_Exp)
    Step_RMSE=np.zeros([Num_Exp,n_steps_out])
    
    model = Sequential()
    model.add(LSTM(100, activation='relu',input_shape=(n_steps_in,n_features)))
    model.add(RepeatVector(n_steps_out))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1,activation='relu')))
    model.compile(optimizer='adam', loss='mse')
    model.summary()
    
    start_time=time.time()
    for run in range(Num_Exp):
        print("Experiment",run+1,"in progress")
        # fit model
        model.fit(x_train, y_train, epochs=Iterations,batch_size=64, verbose=0, shuffle=False)
        y_predicttrain = model.predict(x_train)
        y_predicttest = model.predict(x_test)
        train_acc[run] = rmse( y_predicttrain,y_train) 
        test_acc[run] = rmse( y_predicttest, y_test) 
        for j in range(n_steps_out):
            Step_RMSE[run][j]=rmse(y_predicttest[:,j,0], y_test[:,j,0])
            
    print("Total time for",Num_Exp,"experiments",time.time()-start_time)
    return train_acc,test_acc,Step_RMSE



In [4]:
def Plot_Mean(name,TrainRMSE_mean,TestRMSE_mean):
    labels = ['TrainRMSE','TestRMSE']
    MLP_Adam=[TrainRMSE_mean[0],TestRMSE_mean[0]]
    MLP_Sgd=[TrainRMSE_mean[1],TestRMSE_mean[1]]
    LSTM=[TrainRMSE_mean[2],TestRMSE_mean[2]]
    Bi_LSTM=[TrainRMSE_mean[3],TestRMSE_mean[3]]
    EN_DC=[TrainRMSE_mean[4],TestRMSE_mean[4]]
    width = 0.1  # the width of the bars
    Plot(name,labels,width,MLP_Adam,MLP_Sgd,LSTM,Bi_LSTM,EN_DC,"Mean","Train&Test_RMSE_Mean_Comparison")


def Plot_Std(name,TrainRMSE_Std,TestRMSE_Std):
    labels = ['TrainRMSE','TestRMSE']
    MLP_Adam=[TrainRMSE_Std[0],TestRMSE_Std[0]]
    MLP_Sgd=[TrainRMSE_Std[1],TestRMSE_Std[1]]
    LSTM=[TrainRMSE_Std[2],TestRMSE_Std[2]]
    Bi_LSTM=[TrainRMSE_Std[3],TestRMSE_Std[3]]
    EN_DC=[TrainRMSE_Std[4],TestRMSE_Std[4]]
    width = 0.1  # the width of the bars
    Plot(name,labels,width,MLP_Adam,MLP_Sgd,LSTM,Bi_LSTM,EN_DC,"Standard Deviation","Train&Test_RMSE_Std_Comparison")

def Plot_Step_RMSE_Mean(name,Step_RMSE_mean):
    MLP_Adam=Step_RMSE_mean[0,:]
    MLP_Sgd=Step_RMSE_mean[1,:]
    LSTM=Step_RMSE_mean[2,:]
    Bi_LSTM=Step_RMSE_mean[3,:]
    EN_DC=Step_RMSE_mean[4,:]
    labels = []
    for j in range(Step_RMSE_mean.shape[1]):
        labels=np.concatenate((labels,[str(j+1)]))
    width = 0.1  # the width of the bars
    Plot(name,labels,width,MLP_Adam,MLP_Sgd,LSTM,Bi_LSTM,EN_DC,"RMSE_Mean","Step_RMSE_Comparison")
    

def Plot(name,labels,width,MLP_Adam,MLP_Sgd,LSTM,Bi_LSTM,EN_DC,typ,Gname):
    r1 = np.arange(len(labels))
    r2 = [x + width for x in r1]
    r3 = [x + width for x in r2]
    r4 = [x + width for x in r3]
    r5 = [x + width for x in r4]

    fig, ax = plt.subplots()
    rects1 = ax.bar(r1, MLP_Adam, width, label='MLP_Adam')
    rects2 = ax.bar(r2, MLP_Sgd, width, label='MLP_Sgd')
    rects3 = ax.bar(r3, LSTM, width, label='LSTM')
    rects4 = ax.bar(r4, Bi_LSTM, width, label='Bi_LSTM')
    rects5 = ax.bar(r5, EN_DC, width, label='EN_DC')

    plt.ylabel(typ)
    ax.set_title('MLP_Adam vs MLP_Sgd vs Stacked LSTM vs Bi-LSTM vs EN-DC LSTM')
    plt.xticks([r + width for r in range(len(MLP_Adam))], labels)
    ax.legend()
    fig.tight_layout()
    plt.savefig("Results/"+name+"/"+Gname+".png",dpi=100)
    plt.show()
    

In [ ]:
def main():
    
    n_steps_in, n_steps_out = 5,10
    
    Overall_Analysis=np.zeros([35,6+n_steps_out*3])
    for i in range(1,8):
        problem=i
        if problem ==1:
            TrainData = pd.read_csv("../data/Lazer/train1.csv",index_col = 0)
            TrainData = TrainData.values
            TestData = pd.read_csv("../data/Lazer/test1.csv",index_col = 0)
            TestData = TestData.values
            name= "Lazer"
        if problem ==2:
            TrainData = pd.read_csv("../data/Sunspot/train1.csv",index_col = 0)
            TrainData = TrainData.values
            TestData = pd.read_csv("../data/Sunspot/test1.csv",index_col = 0)
            TestData = TestData.values
            name= "Sunspot"
        if problem ==3:
            TrainData = pd.read_csv("../data/Mackey/train1.csv",index_col = 0)
            TrainData = TrainData.values
            TestData = pd.read_csv("../data/Mackey/test1.csv",index_col = 0)
            TestData = TestData.values
            name="Mackey"
        if problem ==4:
            TrainData = pd.read_csv("../data/Lorenz/train1.csv",index_col = 0)
            TrainData = TrainData.values
            TestData = pd.read_csv("../data/Lorenz/test1.csv",index_col = 0)
            TestData = TestData.values  
            name= "Lorenz"
        if problem ==5:
            TrainData = pd.read_csv("../data/Rossler/train1.csv",index_col = 0)
            TrainData = TrainData.values
            TestData = pd.read_csv("../data/Rossler/test1.csv",index_col = 0)
            TestData = TestData.values
            name= "Rossler"
        if problem ==6:
            TrainData = pd.read_csv("../data/Henon/train1.csv",index_col = 0)
            TrainData = TrainData.values
            TestData = pd.read_csv("../data/Henon/test1.csv",index_col = 0)
            TestData = TestData.values
            name= "Henon"
        if problem ==7:
            TrainData = pd.read_csv("../data/ACFinance/train1.csv",index_col = 0)
            TrainData = TrainData.values
            TestData = pd.read_csv("../data/ACFinance/test1.csv",index_col = 0)
            TestData = TestData.values
            name= "ACFinance" 

        x_train = TrainData[:,0:n_steps_in]
        y_train = TrainData[:,n_steps_in : n_steps_in+n_steps_out ]
        x_test = TestData[:,0:n_steps_in]
        y_test = TestData[:,n_steps_in : n_steps_in+n_steps_out]

        print(name)
        Num_Exp=5    #No. of experiments
        Iterations=500
        TrainRMSE_mean=np.zeros(5)
        TestRMSE_mean=np.zeros(5)
        TrainRMSE_Std=np.zeros(5)
        TestRMSE_Std=np.zeros(5)
        Step_RMSE_mean=np.zeros([5,n_steps_out])
        train_acc=np.zeros(Num_Exp)
        test_acc=np.zeros(Num_Exp)
        Step_RMSE=np.zeros([Num_Exp,n_steps_out])


        for k in range(1,6):

            method=k
            if method ==1:
                train_acc,test_acc,Step_RMSE=MODEL_FNN_adam(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations)
                Mname="MODEL_FNN_adam"
            if method ==2:
                train_acc,test_acc,Step_RMSE=MODEL_FNN_sgd(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations)
                Mname="MODEL_FNN_sgd"
            if method ==3:
                train_acc,test_acc,Step_RMSE=MODEL_LSTM(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations)
                Mname="MODEL_LSTM"
            if method ==4:
                train_acc,test_acc,Step_RMSE=MODEL_Bi_LSTM(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations)
                Mname="MODEL_Bi_LSTM"
            if method ==5:
                train_acc,test_acc,Step_RMSE=MODEL_EN_DC(x_train,x_test,y_train,y_test,Num_Exp,n_steps_in,n_steps_out,Iterations)
                Mname="MODEL_EN_DC"

            print(Mname)

            Overall_Analysis[(i-1)*5+(k-1)][0]=np.mean(train_acc)
            Overall_Analysis[(i-1)*5+(k-1)][1]=np.std(train_acc)
            Overall_Analysis[(i-1)*5+(k-1)][2]=np.min(train_acc)
            Overall_Analysis[(i-1)*5+(k-1)][3]=np.mean(test_acc)
            Overall_Analysis[(i-1)*5+(k-1)][4]=np.std(test_acc)
            Overall_Analysis[(i-1)*5+(k-1)][5]=np.min(test_acc)

            arr = np.dstack((train_acc,test_acc))
            arr=arr.reshape(Num_Exp,2)
            arr=np.concatenate((arr,Step_RMSE), axis=1)
            arr=arr.reshape(Num_Exp,2+n_steps_out)
            ExpIndex=np.array([])
            for j in range(Num_Exp):
                ExpIndex=np.concatenate((ExpIndex,["Exp"+str(j+1)]))
            
            TrainRMSE_mean[k-1]=np.mean(train_acc)
            TestRMSE_mean[k-1]=np.mean(test_acc)
            TrainRMSE_Std[k-1]=np.std(train_acc)
            TestRMSE_Std[k-1]=np.std(test_acc)
            
            ExpIndex1=['TrainRMSE','TestRMSE']
            for j in range(n_steps_out):
                Step_RMSE_mean[k-1][j]=np.mean(Step_RMSE[:,j])
                ExpIndex1=np.concatenate((ExpIndex1,["Step"+str(j+1)]))
            arr = pd.DataFrame(arr, index = ExpIndex , columns = ExpIndex1)
            arr.to_csv("Results/"+name+"/"+Mname+"/ExpAnalysis.csv")
            print(arr)
            
            arr1 = np.vstack(([np.mean(train_acc),np.std(train_acc),np.min(train_acc),np.max(train_acc)],[np.mean(test_acc), np.std(test_acc),np.min(test_acc),np.max(test_acc)]))
            for j in range(n_steps_out):
                Step_mean = np.mean(Step_RMSE[:,j])
                Step_std = np.std(Step_RMSE[:,j])
                Step_min = np.min(Step_RMSE[:,j])
                arr1=np.vstack((arr1,[Step_mean,Step_std,Step_min,np.max(Step_RMSE[:,j])]))
                Overall_Analysis[(i-1)*5+(k-1)][3*j+6]= Step_mean
                Overall_Analysis[(i-1)*5+(k-1)][3*j+7]= Step_std
                Overall_Analysis[(i-1)*5+(k-1)][3*j+8]= Step_min
            arr1 = pd.DataFrame(arr1, index=ExpIndex1, columns = ['Mean','Standard Deviation','Min','Max'])
            print(arr1)
            arr1.to_csv("Results/"+name+"/"+Mname+"/OverallAnalysis.csv")
        
        
        
        #Plot mean of train_RMSE and test_RMSE
        Plot_Mean(name,TrainRMSE_mean,TestRMSE_mean)
        #Plot Std of train_RMSE and test_RMSE
        Plot_Std(name,TrainRMSE_Std,TestRMSE_Std)
        #Plot Step wise RMSE mean for different methods
        Plot_Step_RMSE_Mean(name,Step_RMSE_mean)

        
    Index1=[]
    for j in range(7):
        Index1=np.concatenate((Index1, ['MLP_Adam','MLP_Sgd','Stacked_LSTM','Bi-LSTM','EN-DC LSTM']))
    Index2=["Lazer","Sunspot","Mackey","Lorenz","Rossler","Henon","ACFinance"]
    Index2=np.repeat(Index2,5)
    Index=np.dstack((Index2,Index1))
    Index=Index.reshape(35,2)
    Column=['Dataset','Method','TrainRMSE_Mean','TrainRMSE_Std','TrainRMSE_Min','TestRMSE_Mean','TestRMSE_Std','TestRMSE_Min']
    for j in range(1,11):
        Column=np.concatenate((Column, ['Step'+str(j)+'RMSE_Mean','Step'+str(j)+'RMSE_Std','Step'+str(j)+'RMSE_Min']))
  
    Overall_Analysis=np.concatenate((Index,Overall_Analysis), axis=1)
    Overall_Analysis = pd.DataFrame(Overall_Analysis, columns = Column)
    print(Overall_Analysis)
    Overall_Analysis.to_csv("Results/OverallAnalysis.csv")
    
    
    
if __name__ == "__main__": main()

Lazer
Experiment 1 in progress
Experiment 2 in progress
Experiment 3 in progress
Experiment 4 in progress
Experiment 5 in progress
Total time for 5 experiments 1.0189318656921387
MODEL_FNN_adam
      TrainRMSE  TestRMSE     Step1     Step2     Step3     Step4     Step5  \
Exp1   0.325443  0.239534  0.071267  0.103195  0.069258  0.074398  0.073776   
Exp2   0.321768  0.240550  0.072885  0.102573  0.073675  0.069117  0.071709   
Exp3   0.322768  0.243118  0.073054  0.103178  0.074966  0.075403  0.074598   
Exp4   0.323223  0.238217  0.076227  0.098575  0.069978  0.074229  0.071913   
Exp5   0.319841  0.227785  0.066930  0.092599  0.069761  0.073901  0.070693   

         Step6     Step7     Step8     Step9    Step10  
Exp1  0.064254  0.060956  0.063584  0.083775  0.083476  
Exp2  0.068553  0.057678  0.060564  0.086732  0.086411  
Exp3  0.064314  0.059335  0.061509  0.083824  0.088242  
Exp4  0.066381  0.058064  0.065208  0.082624  0.082371  
Exp5  0.061187  0.058670  0.060652  0.076384  

/Users/shauryagoyal32/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Experiment 2 in progress


/Users/shauryagoyal32/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Experiment 3 in progress


...

Experiment 4 in progress


/Users/shauryagoyal32/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Experiment 5 in progress


/Users/shauryagoyal32/opt/anaconda3/envs/tensorflow/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Total time for 5 experiments 3.6160831451416016
MODEL_FNN_sgd
      TrainRMSE  TestRMSE     Step1     Step2     Step3     Step4     Step5  \
Exp1   0.387590  0.253036  0.085767  0.087362  0.077185  0.077057  0.075035   
Exp2   0.380884  0.251639  0.082969  0.091027  0.078226  0.076111  0.075468   
Exp3   0.382714  0.251165  0.073802  0.089071  0.075044  0.081060  0.083445   
Exp4   0.388324  0.256655  0.074947  0.090269  0.069331  0.084774  0.078178   
Exp5   0.381530  0.251061  0.083064  0.088865  0.070628  0.078463  0.073557   

         Step6     Step7     Step8     Step9    Step10  
Exp1  0.073816  0.069720  0.076873  0.093243  0.081244  
Exp2  0.071641  0.059816  0.077590  0.096541  0.080584  
Exp3  0.072665  0.063926  0.075402  0.092981  0.082707  
Exp4  0.071111  0.065847  0.078043  0.096553  0.095892  
Exp5  0.072762  0.064685  0.072837  0.097490  0.085973  
               Mean  Standard Deviation       Min       Max
TrainRMSE  0.384209            0.003125  0.380884  0.388324
T